In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import gzip
from multiprocessing import Pool
import os
import pandas as pd 
import re



In [2]:
import os

In [3]:
vcf_file = '/mnt/h700/omixlab/workflows/bipin_works/VCF_ANALYSIS_APR_HPRC_CPR/arp-v2.1-mc-chm13_single_allelic.vcf'


In [4]:
import pickle

In [5]:
def extract_info(info_str, key):
    for field in info_str.split(';'):
        if field.startswith(key + '='):
            return field[len(key)+1:]
    return None

In [7]:
def allele_lengths(ref, alt):
    alts = alt.split(',')
    return [len(a) for a in alts]

In [12]:
def split_string(s):
    result = re.split('(>|<)', s)
    result = ''.join([item for item in result if item][:2])
    return result

In [11]:
def split_info(s):
    string = s
    result = string.split(';')
    at = result[3]
    at_new= '='.join([at.split('=')[0]] + [''.join([item for item in re.split('(>|<)', at.split('=')[-1]) if item][:2])])
    return_string = ';'.join(result[:3] + [at_new] + result[4:])
    return return_string

In [18]:
all_sites = []
with open(vcf_file, 'r') as vcf:
    for line in vcf:
        if not line.startswith('#'):
            columns = line.strip().split('\t')
            id = columns[2]
            info = columns[7]
            
            columns[2] = split_string(columns[2])
            columns[7] = split_info(columns[7])
            
            
            all_sites.append('\t'.join(columns))
            
        

['##fileformat=VCFv4.2',
 '##FILTER=<ID=PASS,Description="All filters passed">',
 '##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">',
 '##INFO=<ID=CONFLICT,Number=.,Type=String,Description="Sample names for which there are multiple paths in the graph with conflicting alleles">',
 '##INFO=<ID=AC,Number=A,Type=Integer,Description="Total number of alternate alleles in called genotypes">',
 '##INFO=<ID=AF,Number=A,Type=Float,Description="Estimated allele frequency in the range (0,1]">',
 '##INFO=<ID=NS,Number=1,Type=Integer,Description="Number of samples with data">',
 '##INFO=<ID=AN,Number=1,Type=Integer,Description="Total number of alleles in called genotypes">',
 '##INFO=<ID=LV,Number=1,Type=Integer,Description="Level in the snarl tree (0=top level)">',
 '##INFO=<ID=PS,Number=1,Type=String,Description="ID of variant corresponding to parent snarl">',
 '##INFO=<ID=AT,Number=R,Type=String,Description="Allele Traversal as path in graph">',
 '##contig=<ID=chr1,length=248387328>',

In [19]:
comments = []
with open(vcf_file, 'r') as vcf:
    for line in vcf:
        if line.startswith('#'):
            comments.append(line.strip())
        else:
            break
            

In [21]:
with open('simplified_single_allelic.vcf', 'w+') as f:
    for comment in comments:
        f.write(f'{comment}\n')
    for site in all_sites:
        f.write(f'{site}\n')

In [9]:
all_sites_df = pd.DataFrame(all_sites)

In [32]:
split_info('>132>123')

IndexError: list index out of range

In [33]:
all_sites_df[7] = all_sites_df[7].apply(split_info)

In [35]:
all_sites_df.iloc[]

'AT=>2106'

In [13]:
interesting_alleles = []
with open(vcf_file, 'r') as vcf:
    for line in vcf:
        if not line.startswith('#'):  # Exclude header lines
            columns = line.strip().split('\t')
            ref = columns[3]
            alt = columns[4]
            
            lengths = allele_lengths(ref, alt)

            if ((len(lengths) >= 5 and any((l-len(ref)) > 10000 for l in lengths)) ):
                interesting_alleles.append(line.strip())


In [44]:
with open('interesting_sites.tsv' , 'w') as f:
    f.write('chr\tpos\talt\n')
    for site in interesting_alleles:
        items = site.split('\t')
        sv_lengths = [str(len(item)) for item in items[4].split(',')]
        f.write(f'{items[0]}\t{items[1]}\t{",".join((sv_lengths))}\n')

In [8]:
import pandas as pd

In [18]:
def get_df(sites):
    data = []
    for site in sites:
        data.append((site.split()[0], int(site.split()[1])))
     
    return pd.DataFrame(data, columns=['chrom', 'pos'])

In [23]:

all_sites_df[1] = pd.to_numeric(all_sites_df[1])

In [24]:
site_df = get_df(interesting_alleles)

In [50]:
window_size = 5000
regions = []
for i,site in site_df.iterrows():

    chrom = site['chrom']
    pos = site['pos']

    sites_in_window = all_sites_df.loc[(all_sites_df[0] == chrom ) & (all_sites_df[1]>pos-window_size) & (all_sites_df[1]<pos+window_size)]
    
    if sites_in_window.shape[0] >= 2:
        regions.append([chrom, pos-window_size, pos+window_size])
        
       
    
    
        

In [51]:
len(regions)

386

In [45]:
def extract_genes_from_gff3(filename):
    genes = []
    with open(filename, 'r') as gff:
        for line in gff:
            if not line.startswith("#"):  # Ignore comment lines
                parts = line.strip().split("\t")
                if parts[2] == "gene" or parts[2] == "pseudogene":
                    chrom = parts[0]
                    start = int(parts[3])
                    end = int(parts[4])
                    attributes = parts[8].split(";")
                    gene_name = None
                    is_protein_coding = False
                    for attribute in attributes:
                        if attribute.startswith("ID="):
                            gene_name = attribute.split("=")[1]
                        if attribute == "gene_biotype=protein_coding":
                                is_protein_coding = True
                            
                    if gene_name:
                        genes.append((chrom,start, end, '-'.join(gene_name.split('-')[1:]), is_protein_coding))
    return genes


In [52]:
def variants_within_flank(variants, genes, flank_size):
    result = {}
    
    for i,line in variants.iterrows():
        chrom = line['CHROM']
        pos = int(line['POS'])
        start_flank = pos - flank_size
        end_flank = pos + flank_size
        
        # Check if genes are within the flanking region
        for start, end, gene_name in genes.get(chrom, []):
                    if start < end_flank and end > start_flank:
                        key = (chrom, pos)
                        if key not in result:
                            result[key] = []
                        result[key].append(gene_name)
    return result

In [34]:
def count_genes(gff3_file):
    with open(gff3_file, 'r') as f:
        # Filter out comment lines and split by tabs
        lines = [line.split('\t') for line in f if not line.startswith('#')]
        
        # Count lines where the third column is "gene"
        return sum(1 for line in lines if len(line) > 2 and line[2] == 'gene')

In [46]:
def extract_genes_from_region(filename, chrom_filter, start_filter, end_filter):
    all_genes = []
    pc_genes = []
    with open(filename, 'r') as gff:
        for line in gff:
            if not line.startswith("#"):  # Ignore comment lines
                parts = line.strip().split("\t")
                chrom = parts[0]
                start = int(parts[3])
                end = int(parts[4])
                
                # Filter for the specific region
                if chrom == chrom_filter and (parts[2] == "gene" or parts[2] == "pseudogene") and start_filter <= start and end_filter >= end:
                    attributes = parts[8].split(";")
                    gene_name = None
                    is_protein_coding = False
                    for attribute in attributes:
                        if attribute.startswith("ID="):
                            gene_name = attribute.split("=")[1]
                        if attribute == "gene_biotype=protein_coding":
                                is_protein_coding = True
                                
                        
                    if gene_name:
                        all_genes.append(gene_name)
                        
                    if gene_name and is_protein_coding:
                        pc_genes.append(gene_name)
    return all_genes, pc_genes

In [47]:
def extract_genes_from_region(genes, chrom_filter, start_filter, end_filter):
    all_genes = []
    pc_genes = []
   
    for line in gff:
        if not line.startswith("#"):  # Ignore comment lines
            parts = line.strip().split("\t")
            chrom = parts[0]
            start = int(parts[3])
            end = int(parts[4])
            
            # Filter for the specific region
            if chrom == chrom_filter and (parts[2] == "gene" or parts[2] == "pseudogene") and start_filter <= start and end_filter >= end:
                attributes = parts[8].split(";")
                gene_name = None
                is_protein_coding = False
                for attribute in attributes:
                    if attribute.startswith("ID="):
                        gene_name = attribute.split("=")[1]
                    if attribute == "gene_biotype=protein_coding":
                            is_protein_coding = True
                            
                    
                if gene_name:
                    all_genes.append(gene_name)
                    
                if gene_name and is_protein_coding:
                    pc_genes.append(gene_name)
    return all_genes, pc_genes

In [53]:

gff3_file = "chm13_genomic_chr.gff3"

all_genes_df = pd.DataFrame(extract_genes_from_gff3(gff3_file))
# total_genes = count_genes(gff3_file)

In [54]:
recessive_genes = pd.read_excel('Recessive genes_GA gene list.xlsx')

In [55]:
recessive_gene_set = set(recessive_genes['Gene Name'])

In [56]:
for region in regions:
    chr = region[0]
    start = region[1]
    end = region[2]

    temp_genes_df = all_genes_df.loc[(all_genes_df[0] == chr) & (pd.to_numeric(all_genes_df[1]) > start) & (pd.to_numeric(all_genes_df[2]) < end)   ]
    all = list(temp_genes_df[3])
    pc = list(temp_genes_df.loc[temp_genes_df[4], 3])

    region.append(','.join(all))
    region.append(len(all))
    region.append(','.join(pc))
    region.append(len(pc))
    region.append(''.join(set(pc).intersection(recessive_gene_set)))
    region.append(len(set(pc).intersection(recessive_gene_set)))
    

In [57]:
pd.DataFrame(regions, columns=['chr','start','end', 'all_genes','no of genes', 'protein_genes', 'no of pc genes', 'recessive_genes', 'no of recessive genes']).to_csv('genes_in_regions_with_one_extra_site_10kb.csv', index=None)

In [117]:
set('RPS9,LILRB3,LILRA6,LILRB5,LILRB2'.split(',')).intersection(recessive_gene_set)

set()

In [104]:
range = 500000

In [105]:
result = variants_within_flank(df, genes, range)

In [4]:
rows = [(key, val) for key, values in result.items() for val in values]

# Create a DataFrame
gene_df = pd.DataFrame(rows, columns=['loc', 'gene'])


recessive_genes = pd.read_excel('Recessive genes_GA gene list.xlsx')
intersect = gene_df.merge(recessive_genes, how='left', left_on='gene', right_on='Gene Name').dropna().drop(['Gene Name'], axis=1)
intersect_dict = {}
for i, row in intersect.iterrows():
    if row['loc'] not in intersect_dict:
        intersect_dict[row['loc']] = [] 
    intersect_dict[row['loc']].append(str(row['gene']))


NameError: name 'result' is not defined

In [107]:
# intersect.to_csv('gene_recessive_intersection.csv', index= None)
# intersect.groupby('loc').count().sort_values('gene', ascending=False).to_csv('count_recessive.csv', index=None)

In [108]:
from scipy.stats import fisher_exact
def pvalue_and_odds_ratio(total, first_set, second_set, overlap):
    table = [[overlap, first_set-overlap], [second_set-overlap, total-first_set-second_set+overlap]]
    odds_ratio, p_value = fisher_exact(table)
    return odds_ratio, p_value

In [109]:
pvalues= []
for k,v in result.items():
    site_genes = len(set(v))
    if k in intersect_dict:
        intersected_genes = len(set(intersect_dict[k]))
    else:
        intersected_genes = 0
    total_recessive = recessive_genes.shape[0]
    
    o, p = pvalue_and_odds_ratio(total_genes, site_genes, total_recessive, intersected_genes)
    
    
    if k in intersect_dict:
        pvalues.append((k, p, o,site_genes,intersected_genes,','.join(set(v)), ','.join(set(intersect_dict[k]))))
    else:
        pvalues.append((k, p, o,site_genes,intersected_genes,','.join(set(v)), ''))
    
    

In [110]:
p_df = pd.DataFrame(pvalues, columns = ['loc', 'pvalue', 'odds_ratio','number of genes in site', 'number of recessive genes in site', 'all genes', 'genes'])

In [111]:
p_df.to_csv(f'recessive_overlap_{range*2}.csv')

In [112]:
(p_df.loc[p_df['loc']==('chr13', 12990689)]['genes'].str.len())

50    0
Name: genes, dtype: int64

In [113]:
p_df.to_csv('pvalue_per_site.csv', index=None)

In [114]:
intersect['gene'].unique().shape[0]

138

7018

In [ ]:
from scipy.stats import fisher_exact
def pvalue_and_odds_ratio(total, first_set, second_set, overlap):
    table = [[overlap, first_set-overlap], [second_set-overlap, total-first_set-second_set+overlap]]
    odds_ratio, p_value = fisher_exact(table)

In [5]:
duplication_ratio_matrix = pd.read_csv('dup_ratio.csv')
intersect=(set(duplication_ratio_matrix['Gene Name']).intersection(set(recessive_genes['Gene Name'])))

In [6]:
duplication_ratio_matrix['sum'] = duplication_ratio_matrix[duplication_ratio_matrix['Gene Name'].isin(intersect)].iloc[:, 1:].sum(axis=1)
duplication_ratio_matrix=duplication_ratio_matrix.sort_values('sum', ascending=False)

In [7]:
duplication_ratio_matrix.iloc[:10]

,Gene Name,brother-brother/1,brother-brother/2,father-father/1,father-father/2,gp001-gp001/1,gp001-gp001/2,gp002-gp002/1,gp002-gp002/2,gp003-gp003/1,...,gp038-gp038/2,gp039-gp039/1,gp039-gp039/2,gp040-gp040/1,gp040-gp040/2,gp041-gp041/1,gp041-gp041/2,mother-mother/1,mother-mother/2,sum
1030,RRP7A,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0
1094,SMN1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,9.0
244,CYP21A2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
251,DBR1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
297,DPY19L2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
417,FDFT1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0
1415,ZP2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0
107,C4B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
1157,SYCE1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
880,PHKG2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


In [41]:
def get_gene_location_from_gff(gff_file, gene_names):
    """
    Extract the location of a gene from a GFF file.
    
    Args:
    - gff_file (str): Path to the GFF file.
    - gene_name (str): Name of the gene.
    
    Returns:
    - dict: Information about the gene including its location, or None if not found.
    """
    
    found_genes = []
    with open(gff_file, 'r') as file:
        for line in file:
            if line.startswith('#'):  # Skip comment lines
                continue
            fields = line.strip().split('\t')
            if len(fields) < 9:  # A valid GFF row should have 9 columns
                continue
            
            # Check if the current feature is a gene and if the name matches
            feature_type = fields[2]
            attributes = fields[8]
   

            gene_name = '-'.join(attributes.split(';')[0].split('=')[-1].split('-')[1:])
           
            
            if feature_type == "gene" and gene_name in genes:
                found_genes.append( {
                    "Gene": gene_name,
                    "Chromosome": fields[0],
                    "Start": int(fields[3]),
                    "End":int(fields[4]),
                    "Strand": fields[6]
                }
            )
    return found_genes



In [42]:

genes = list(duplication_ratio_matrix['Gene Name'].iloc[:10])
gff_path = "chm13_genomic_chr.gff3"
gene_name = 'RRP7A'
genes_list = get_gene_location_from_gff(gff_path, gene_name)



In [43]:
genes_list

[{'Gene': 'DBR1',
  'Chromosome': 'chr3',
  'Start': 140901580,
  'End': 140915511,
  'Strand': '-'},
 {'Gene': 'SMN1',
  'Chromosome': 'chr5',
  'Start': 71381729,
  'End': 71423141,
  'Strand': '+'},
 {'Gene': 'C4B',
  'Chromosome': 'chr6',
  'Start': 31868001,
  'End': 31888625,
  'Strand': '+'},
 {'Gene': 'CYP21A2',
  'Chromosome': 'chr6',
  'Start': 31891622,
  'End': 31894848,
  'Strand': '+'},
 {'Gene': 'FDFT1',
  'Chromosome': 'chr8',
  'Start': 7900999,
  'End': 7944714,
  'Strand': '-'},
 {'Gene': 'SYCE1',
  'Chromosome': 'chr10',
  'Start': 134505424,
  'End': 134519736,
  'Strand': '-'},
 {'Gene': 'DPY19L2',
  'Chromosome': 'chr12',
  'Start': 63537270,
  'End': 63646941,
  'Strand': '-'},
 {'Gene': 'ZP2',
  'Chromosome': 'chr16',
  'Start': 21128399,
  'End': 21145450,
  'Strand': '-'},
 {'Gene': 'PHKG2',
  'Chromosome': 'chr16',
  'Start': 31135424,
  'End': 31148597,
  'Strand': '+'},
 {'Gene': 'RRP7A',
  'Chromosome': 'chr22',
  'Start': 42989327,
  'End': 43000778,
  '

In [50]:
def parse_gff(gff_file):
    """Extract all genes and their positions from a GFF file."""
    genes = []
    with open(gff_file, 'r') as file:
        for line in file:
            if line.startswith('#'):  # Skip comment lines
                continue
            fields = line.strip().split('\t')
            if len(fields) < 9:  # A valid GFF row should have 9 columns
                continue
            
            # Check if the current feature is a gene
            feature_type = fields[2]
            attributes = fields[8]
            gene_name = '-'.join(attributes.split(';')[0].split('=')[-1].split('-')[1:])
            if feature_type == "gene":
                genes.append({
                    'Gene': fields[8].split(';')[0].split('=')[-1],
                    "Chromosome": fields[0],
                    "Start": int(fields[3]),
                    "End": int(fields[4]),
                    "Strand": fields[6]
                })
    return genes

def find_nearby_genes(target_gene, gff_genes, distance=10000):
    """Find genes within a specified distance from a target gene."""
    nearby = []
    for gene in gff_genes:
        if gene["Chromosome"] == target_gene["Chromosome"]:
            # Check if gene is within the distance upstream or downstream
            if target_gene["Start"] - distance <= gene["End"] <= target_gene["Start"] or target_gene["End"] <= gene["Start"] <= target_gene["End"] + distance:
                nearby.append(gene)
    return nearby

In [51]:
gff_genes = parse_gff(gff_path)

In [52]:
for gene in genes_list:
    nearby_genes = find_nearby_genes(gene, gff_genes)
    
    gene['nearby_genes'] = nearby_genes
    for nearby in nearby_genes:
        print(f"- {nearby['Gene']} {nearby['Chromosome']}:{nearby['Start']}-{nearby['End']} (Strand: {nearby['Strand']})")
    print("\n")



- gene-SERF1A chr5:71357441-71375322 (Strand: +)
- gene-NAIP chr5:71424931-71482091 (Strand: -)


- gene-CYP21A2 chr6:31891622-31894848 (Strand: +)
- gene-TNXB chr6:31894357-31962530 (Strand: -)


- gene-C4B chr6:31868001-31888625 (Strand: +)


- gene-CTSB chr8:7872212-7897789 (Strand: +)
- gene-NEIL2 chr8:7952965-7970597 (Strand: -)




- gene-LOC105369797 chr12:63524154-63530248 (Strand: -)




- gene-SRCAP chr16:31086152-31128418 (Strand: +)
- gene-TMEM265 chr16:31127651-31132204 (Strand: +)
- gene-RNF40 chr16:31149033-31163750 (Strand: +)


- gene-LOC101927372 chr22:42982889-42986839 (Strand: -)




In [54]:
sorted(genes_list, key = lambda x: len(x['nearby_genes']), reverse=True)

[{'Gene': 'PHKG2',
  'Chromosome': 'chr16',
  'Start': 31135424,
  'End': 31148597,
  'Strand': '+',
  'nearby_genes': [{'Gene': 'gene-SRCAP',
    'Chromosome': 'chr16',
    'Start': 31086152,
    'End': 31128418,
    'Strand': '+'},
   {'Gene': 'gene-TMEM265',
    'Chromosome': 'chr16',
    'Start': 31127651,
    'End': 31132204,
    'Strand': '+'},
   {'Gene': 'gene-RNF40',
    'Chromosome': 'chr16',
    'Start': 31149033,
    'End': 31163750,
    'Strand': '+'}]},
 {'Gene': 'SMN1',
  'Chromosome': 'chr5',
  'Start': 71381729,
  'End': 71423141,
  'Strand': '+',
  'nearby_genes': [{'Gene': 'gene-SERF1A',
    'Chromosome': 'chr5',
    'Start': 71357441,
    'End': 71375322,
    'Strand': '+'},
   {'Gene': 'gene-NAIP',
    'Chromosome': 'chr5',
    'Start': 71424931,
    'End': 71482091,
    'Strand': '-'}]},
 {'Gene': 'C4B',
  'Chromosome': 'chr6',
  'Start': 31868001,
  'End': 31888625,
  'Strand': '+',
  'nearby_genes': [{'Gene': 'gene-CYP21A2',
    'Chromosome': 'chr6',
    'Start'

In [22]:
genes_list

[]